In [ ]:
# uncomment these two lines if using a gpu for a speedup
#! nvidia-smi
#os.environ["CUDA_VISIBLE_DEVICES"] = "0"

from desc import set_device
set_device("gpu") 
# ! kill -9 7274
! nvidia-smi

from desc import equilibrium

# imports

import numpy as np
from desc.coils import CoilSet, FourierPlanarCoil
import desc.examples
from desc.equilibrium import Equilibrium
from desc.plotting import plot_surfaces, plot_2d, plot_3d, plot_coils
from desc.grid import LinearGrid, ConcentricGrid
from desc.coils import MixedCoilSet
from desc.objectives import (
    ObjectiveFunction,
    #coil
    CoilCurvature,
    CoilLength,
    CoilTorsion,
    CoilSetMinDistance,
    PlasmaCoilSetMinDistance,
    QuadraticFlux,
    ToroidalFlux,
    FixCoilCurrent,
    FixParameters,
    # plasma
    FixBoundaryR,
    FixBoundaryZ,
    FixPressure,
    FixCurrent,
    FixIota,
    FixPsi,
    AspectRatio,
    ForceBalance,
    QuasisymmetryBoozer,
    QuasisymmetryTwoTerm,
    QuasisymmetryTripleProduct,
    VacuumBoundaryError,
)
from desc.continuation import solve_continuation_automatic
from desc.optimize import Optimizer
from desc.magnetic_fields import field_line_integrate
#from desc.singularities import compute_B_plasma
import time
import plotly.express as px
import plotly.io as pio

from desc.geometry import FourierRZToroidalSurface


# This ensures Plotly output works in multiple places:
# plotly_mimetype: VS Code notebook UI
# notebook: "Jupyter: Export to HTML" command in VS Code
# See https://plotly.com/python/renderers/#multiple-renderers
pio.renderers.default = "plotly_mimetype+notebook"

# plotting
from desc.plotting import (
    plot_grid,
    plot_boozer_modes,
    plot_boozer_surface,
    plot_qs_error,
    plot_boundaries,
    plot_boundary,
)

In [ ]:
eq_init = desc.examples.get("precise_QA")

In [ ]:
r0 = eq_init.compute(["R0"])["R0"]
a0 = eq_init.compute("a")["a"]
coil_surface = FourierRZToroidalSurface(
    R_lmn=[r0, a0*3.5, 0],
    Z_lmn=[a0*3.5, 0],
    modes_R=[[0, 0], [1, 0], [0, 1]],
    modes_Z=[[-1, 0], [0, -1]],
    NFP=1,
)
plot_grid = LinearGrid(M=20, N=40, NFP=1, endpoint=True)

eq_coil_surf = Equilibrium(M=4, N=4, Psi=0.1, surface=coil_surface)
# fig = plot_3d(eq_init, "|B|", grid=plot_grid)
# new = plot_3d(eq_coil_surf, "rho", fig = fig, grid=plot_grid, alpha = 0.5)

# fig.show()

In [ ]:
##############################
# Generating surface coils #
##############################

# minor_radius = eq_init.compute("a")["a"]
# surface = eq_init.surface
surface = coil_surface

grid = LinearGrid(M=8, N=8, NFP=surface.NFP, sym=eq_init.sym, rho = 1)

centers = surface.compute(["x"], grid=grid)["x"]
normals = surface.compute(["n_rho"], grid=grid)["n_rho"]

num_coils = len(centers)

surface_coils = []
for k in range(num_coils):
    coil = FourierPlanarCoil(
        current=1e4,
        center=centers[k],
        normal=normals[k],
        r_n=0.2*eq_init.compute("a")["a"],
        basis="rpz",  # we are giving the center and normal in cylindrical coordinates
    ).to_FourierXYZ(
        N=2
    ) 
    surface_coils.append(coil)

surf_coilset = CoilSet(surface_coils, NFP=eq_init.NFP, sym=eq_init.sym)

## to plot surface coils:

# plot_grid = LinearGrid(M=20, N=40, NFP=1, endpoint=True)
# fig = plot_3d(eq_init, "|B|", grid=plot_grid)
# fig = plot_coils(surf_coilset, fig=fig)
# fig.show()

In [ ]:
##############################
# Generating toroidal coils #
##############################

minor_radius = eq_coil_surf.compute("a")["a"]
offset = 0.2
num_coils = 4  # coils per half field period

zeta = np.linspace(0, np.pi / eq_coil_surf.NFP, num_coils, endpoint=False) + np.pi / (
    2 * eq_coil_surf.NFP * num_coils
)

grid = LinearGrid(rho=[0.0], M=0, zeta=zeta, NFP=eq_coil_surf.NFP)
data = eq_coil_surf.axis.compute(["x", "x_s"], grid=grid, basis="rpz")

centers = data["x"]  # center coils on axis position
normals = data["x_s"]  # make normal to coil align with tangent along axis

tor_coils = []
for k in range(num_coils):
    coil = FourierPlanarCoil(
        current=2.5e5,
        center=centers[k, :],
        normal=normals[k, :],
        r_n=minor_radius + offset,
        basis="rpz",  # we are giving the center and normal in cylindrical coordinates
    ).to_FourierXYZ(
        N=2
    )  # fit with 10 fourier coefficients per coil
    tor_coils.append(coil)

tor_coilset = CoilSet(tor_coils, NFP=eq_coil_surf.NFP, sym=eq_coil_surf.sym)

# visualize the initial coilset
# plot_grid = LinearGrid(M=20, N=40, NFP=1, endpoint=True)
# fig = plot_3d(eq_init, "|B|", grid=plot_grid)
# fig = plot_coils(tor_coilset, fig=fig)
# fig = plot_coils(surf_coilset, fig=fig)

# coilset = MixedCoilSet(tor_coilset, surf_coilset)

# fig.show()

In [ ]:
coilset = MixedCoilSet(tor_coilset, surf_coilset)

# Fixing position and shapes of coils, and only varying currents

coil_indices_to_fix_current = [False for c in coilset]
coil_indices_to_fix_current[0] = True

fixed_params = [
    {'X_n': True, 'Y_n': True, 'Z_n': True, 'current': np.array([0]), 'rotmat': True, 'shift': True}, 
    {'X_n': True, 'Y_n': True, 'Z_n': True, 'current': False, 'rotmat': True, 'shift': True}
]

constraints = (
    FixParameters(coilset, params = fixed_params),
    FixCoilCurrent(coilset, indices=coil_indices_to_fix_current),
)

In [ ]:
# number of points used to discretize coils. This could be different for each objective
# (eg if you want higher resolution for some calculations), but we'll use the same for all of them
coil_grid = LinearGrid(N=50)
# similarly define a grid on the plasma surface where B*n errors will be evaluated
plasma_grid = LinearGrid(M=25, N=25, NFP=eq_init.NFP, sym=eq_init.sym)

# define our objective function
obj = ObjectiveFunction(
    (
        QuadraticFlux(
            eq_init,
            field=coilset,
            # grid of points on plasma surface to evaluate normal field error
            eval_grid=plasma_grid,
            field_grid=coil_grid,
            vacuum=True,  # vacuum=True means we won't calculate the plasma contribution to B as it is zero
            weight=100,
        ),
    )
)

optimizer = Optimizer("lsq-exact")

(optimized_coilset,), _ = optimizer.optimize(
    coilset,
    objective=obj,
    constraints=constraints,
    maxiter=100,
    verbose=3,
    ftol=1e-8,  # stopping tolerance on the function value
    xtol=1e-8,  # stopping tolerance on the step size
    gtol=1e-8,  # stopping tolerance on the gradient
    copy=True,
)

fig = plot_3d(
    eq_init.surface, "B*n", field=optimized_coilset, field_grid=coil_grid, grid=plot_grid
)

fig = plot_coils(optimized_coilset, fig=fig)
fig.show()

In [ ]:
print(max(np.abs(optimized_coilset[1].current)))
print(eq_init.compute("a")["a"])